<a href="https://colab.research.google.com/github/dtnam302/NLP/blob/main/Text_Gen_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Download a book -> style of text gen

In [1]:
!wget https://www.gutenberg.org/cache/epub/21717/pg21717.txt

--2022-03-26 04:35:49--  https://www.gutenberg.org/cache/epub/21717/pg21717.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 157279 (154K) [text/plain]
Saving to: ‘pg21717.txt’

pg21717.txt         100%[===================>] 153.59K  --.-KB/s    in 0.07s   

2022-03-26 04:35:50 (2.16 MB/s) - ‘pg21717.txt’ saved [157279/157279]



In [2]:
file = open('/content/pg21717.txt')
text = file.read()
file.close()
print(text[:500])

﻿The Project Gutenberg EBook of Battles with the Sea, by R.M. Ballantyne

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Battles with the Sea

Author: R.M. Ballantyne

Release Date: June 7, 2007 [EBook #21717]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK BATTLES WITH THE


###Preprocessing using nltk

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Các tokens ở trên đều là các tokens ở dạng thô, do đó để có một training data tốt hơn, chúng ta cần phải thực hiện một số bước Data cleaning như sau:

– Thay dấu ‘-’ và ‘–’ trong các từ như  manne–only, we–especially, land–are, sea-coast, … bằng space để tách các từ này thành các từ độc lập.

– Loại bỏ các dấu câu có trong các từ như: ‘wanting.’,’brave,’,’sea.’,..

– Chỉ giữ lại các từ mà tất cả các ký tự đều là alphabets

– Chuyển tất cả các từ về dạng lowercase

Tham khảo : https://machinelearningmastery.com/clean-text-machine-learning-python/

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# split into words
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)[1:] #remove utf-8 decode
# convert to lower case
tokens = [w.lower() for w in tokens]
# remove punctuation from each word
import string
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]
# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]
# filter out stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
print(words[:100])

['project', 'gutenberg', 'ebook', 'battles', 'sea', 'rm', 'ballantyne', 'ebook', 'use', 'anyone', 'anywhere', 'cost', 'almost', 'restrictions', 'whatsoever', 'may', 'copy', 'give', 'away', 'reuse', 'terms', 'project', 'gutenberg', 'license', 'included', 'ebook', 'online', 'wwwgutenbergorg', 'title', 'battles', 'sea', 'author', 'rm', 'ballantyne', 'release', 'date', 'june', 'ebook', 'language', 'english', 'start', 'project', 'gutenberg', 'ebook', 'battles', 'sea', 'produced', 'nick', 'hodson', 'london', 'england', 'battles', 'sea', 'rm', 'ballantyne', 'chapter', 'one', 'heroes', 'lifeboat', 'rocket', 'skirmishes', 'subject', 'generally', 'ought', 'known', 'english', 'boys', 'terrible', 'costly', 'war', 'british', 'nation', 'times', 'engaged', 'intervals', 'peace', 'mark', 'course', 'war', 'cessations', 'hostilities', 'brief', 'periods', 'treaties', 'peace', 'war', 'knife', 'character', 'quarter', 'neither', 'given', 'sought', 'foe', 'unfeeling', 'unrelenting', 'wastes', 'time', 'diploma

###Build vocab using Keras
Vocab nghĩa là map mỗi từ vựng với index

In [6]:
vocab_size = len(set(words))+1 #token oov
vocab_size

3988

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token="<OOV>")
tokenizer.fit_on_texts(words)

In [ ]:
tokenizer.word_index

In [10]:
word2int_seq = [tokenizer.word_index[word] for word in words]
print('The first ten words: ', words[:10])
print('The corresponding int sequence: ', word2int_seq[:10])

The first ten words:  ['project', 'gutenberg', 'ebook', 'battles', 'sea', 'rm', 'ballantyne', 'ebook', 'use', 'anyone']
The corresponding int sequence:  [7, 36, 201, 309, 8, 667, 668, 201, 160, 428]


Input: 50 từ trước đó của words 
Output: Từ tiếp theo dưới dạng one_hot

In [22]:
import numpy as np
from tensorflow import keras

# Set the sequence length
seq_length=50

# Convert the input data to a list of sequences
inputSequences=list()
outputs=list()

for i in range(seq_length, len(word2int_seq)):
  seq=word2int_seq[i-seq_length:i]
  inputSequences.append(seq)
  outputs.append(word2int_seq[i])
  
# Convert inputSequences to numpy array
inputSequences= np.array(inputSequences)

# One hot encode the output
one_hot_outputs=keras.utils.to_categorical(outputs,num_classes=vocab_size+1) # hàm định nghĩa num_classes = max(outputs) +1

print('Shape of inputSequences: ', inputSequences.shape)
print('Shape of outputs: ', one_hot_outputs.shape)

print('\nThe first input : ')
print('Text format: ',words[:seq_length])
print('Int format: ',inputSequences[0])

print('\nThe Expected output : ')
print('One hot encoding format: ',one_hot_outputs[0])
print('One hot encoding shape: ',one_hot_outputs[0].shape)
print('Int format: ',outputs[0])
print('Text format: ',tokenizer.index_word[outputs[0]])

Shape of inputSequences:  (13267, 50)
Shape of outputs:  (13267, 3989)

The first input : 
Text format:  ['project', 'gutenberg', 'ebook', 'battles', 'sea', 'rm', 'ballantyne', 'ebook', 'use', 'anyone', 'anywhere', 'cost', 'almost', 'restrictions', 'whatsoever', 'may', 'copy', 'give', 'away', 'reuse', 'terms', 'project', 'gutenberg', 'license', 'included', 'ebook', 'online', 'wwwgutenbergorg', 'title', 'battles', 'sea', 'author', 'rm', 'ballantyne', 'release', 'date', 'june', 'ebook', 'language', 'english', 'start', 'project', 'gutenberg', 'ebook', 'battles', 'sea', 'produced', 'nick', 'hodson', 'london']
Int format:  [   7   36  201  309    8  667  668  201  160  428 1236  536   83 1237
 1238   17  140  141   38 1239   67    7   36  119  892  201  669  670
 1240  309    8 1961  667  668 1962  671 1963  201 1964  537  893    7
   36  201  309    8  672 1241 1242  538]

The Expected output : 
One hot encoding format:  [0. 0. 0. ... 0. 0. 0.]
One hot encoding shape:  (3989,)
Int format: 

###Build Model

In [23]:
# Define the dimension of embedding
embedding_dim=50
lstm1_unit=64
lstm2_unit=128
hiden1=128

# Build a model using keras.Sequential()
model=keras.Sequential()

# Add an Embedding layer
model.add(keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=seq_length))

# Add a LSTM layer using CuDNNLSTM for GPU training
# We have to set return_sequences=True for the second LSTM layer
model.add(keras.layers.LSTM(units=lstm1_unit,return_sequences=True))

# Add the second LSTM layer
model.add(keras.layers.LSTM(units=lstm2_unit))

# Add a Dense layer
model.add(keras.layers.Dense(hiden1,activation='relu'))

# Add  the output layer
model.add(keras.layers.Dense(vocab_size+1,activation='softmax')) 

# Display model summary
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            199400    
                                                                 
 lstm_2 (LSTM)               (None, 50, 64)            29440     
                                                                 
 lstm_3 (LSTM)               (None, 128)               98816     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 3989)              514581    
                                                                 
Total params: 858,749
Trainable params: 858,749
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
 
# Start training
model.fit(inputSequences, one_hot_outputs, batch_size=128, epochs=150)

Epoch 1/150


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


104/104 [==============================] - 9s 15ms/step - loss: 7.9398 - accuracy: 0.0110
Epoch 2/150
104/104 [==============================] - 1s 14ms/step - loss: 7.6189 - accuracy: 0.0133
Epoch 3/150
104/104 [==============================] - 1s 14ms/step - loss: 7.4829 - accuracy: 0.0163
Epoch 4/150
104/104 [==============================] - 1s 14ms/step - loss: 7.3229 - accuracy: 0.0215
Epoch 5/150
104/104 [==============================] - 1s 14ms/step - loss: 7.2094 - accuracy: 0.0219
Epoch 6/150
104/104 [==============================] - 1s 14ms/step - loss: 7.1449 - accuracy: 0.0228
Epoch 7/150
104/104 [==============================] - 1s 14ms/step - loss: 7.0337 - accuracy: 0.0216
Epoch 8/150
104/104 [==============================] - 1s 14ms/step - loss: 6.9157 - accuracy: 0.0230
Epoch 9/150
104/104 [==============================] - 1s 14ms/step - loss: 6.7960 - accuracy: 0.0253
Epoch 10/150
104/104 [==============================] - 1s 14ms/step - loss: 6.6852 - accuracy

###Infer

In [33]:
inputSequences[0]

array([   7,   36,  201,  309,    8,  667,  668,  201,  160,  428, 1236,
        536,   83, 1237, 1238,   17,  140,  141,   38, 1239,   67,    7,
         36,  119,  892,  201,  669,  670, 1240,  309,    8, 1961,  667,
        668, 1962,  671, 1963,  201, 1964,  537,  893,    7,   36,  201,
        309,    8,  672, 1241, 1242,  538])

In [42]:
# Input sequence for testing
input_test=inputSequences[0]
 
# Using Model to generate next 50 words
predicted_words=list()
for i in range(50):
  # Get the prediction of the next word (integer number)
  pred = model.predict(np.reshape(input_test,(1,seq_length)), verbose=0)
  
  # Get the next input sequence by appending the predicted word and remove the first word
  input_test= np.append(input_test,np.argmax(pred[0]))
  input_test=input_test[1:]
  
  # Convert integer value to word and append it to the list
  predicted_words.append(tokenizer.index_word[np.argmax(pred[0])])
 
print('The input sequence: ')
print(' '.join(words[:50]))  
 
print('\nThe original words sequence: ')
print(' '.join(words[50:100]))

print('\nThe words sequence generated by the Model: ')
print(' '.join(predicted_words))


The input sequence: 
project gutenberg ebook battles sea rm ballantyne ebook use anyone anywhere cost almost restrictions whatsoever may copy give away reuse terms project gutenberg license included ebook online wwwgutenbergorg title battles sea author rm ballantyne release date june ebook language english start project gutenberg ebook battles sea produced nick hodson london

The original words sequence: 
england battles sea rm ballantyne chapter one heroes lifeboat rocket skirmishes subject generally ought known english boys terrible costly war british nation times engaged intervals peace mark course war cessations hostilities brief periods treaties peace war knife character quarter neither given sought foe unfeeling unrelenting wastes time diplomatic preliminaries scorns

The words sequence generated by the Model: 
england battles sea preserve project electronic electronic works follow copy means exporting copies work web using web specified need official medium forth notice public d

###Improve plan


*   Use pretrain embedding such as Bert
*   Stemming, Lematization 
*   Bigger Dataset or Augumentation

